In [1]:
import pymongo
from pymongo import MongoClient
from pyspark.sql.functions import col, lit, when, from_json
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StringType, IntegerType, ArrayType,BooleanType,StructField,LongType
import pyspark
import pandas as pd
from pyspark.sql import SparkSession
import time

In [2]:
# Grab Currrent Time Before Running the Code
start = time.time()

In [3]:
client = MongoClient("172.23.149.210", 27017)
db = client['cardano_bronze']

In [4]:
# Import the needed tables
reward = db["node.public.reward"]

last_ind = db["last_indexes_2_rewards_history"]

# import required temporary collections to overwrite with new data
reward_tmp = db["reward_temporary"]

In [5]:
# insert initial values in checkpoint table
#initial_val_reward = { "collection": "reward", "last_index": 0 }
#initial_val_pool_hash = { "collection": "pool_hash", "last_index": 0 }
#initial_val_stake_address = { "collection": "stake_address", "last_index": 0 }
#
#last_ind.insert_one(initial_val_reward)
#last_ind.insert_one(initial_val_pool_hash)
#last_ind.insert_one(initial_val_stake_address)

In [6]:
#retrieve the last indices that were processed before
reward_last_processed = last_ind.find_one({'collection': 'reward'})['last_index']

In [8]:
# count how many documents are in each new input mongodb collection
count_reward = reward.estimated_document_count()

In [10]:
# for each Cardano table, select the records which haven't been processed yet (range between last_processed and total records count)
reward_df = reward.find()[reward_last_processed:count_reward]

In [11]:
# drop the previous records in the temporary collections
reward_tmp.drop()

In [12]:
# load the temporary records in the temporary collections
reward_tmp.insert_many(reward_df)

# Initial setup

In [13]:
config = pyspark.SparkConf().setAll([
    #('spark.driver.extraJavaOptions', '-Djava.io.tmpdir=/home/ubuntu/notebook/tmp_spark_env'),
    ('spark.executor.memory', '30g'),
    ('spark.executor.cores', '3'),
    ('spark.cores.max', '3'),
    ('spark.driver.memory','10g'),
    ('spark.executor.instances', '2'),
    ('spark.worker.cleanup.enabled', 'true'),
    ('spark.worker.cleanup.interval', '60'),
    ('spark.worker.cleanup.appDataTtl', '60'),
    ('spark.jars.packages', 'org.mongodb.spark:mongo-spark-connector:10.0.2'),
    ('spark.mongodb.output.writeConcern.wTimeoutMS','120000'),
    ('spark.mongodb.output.writeConcern.socketTimeoutMS','120000'),
    ('spark.mongodb.output.writeConcern.connectTimeoutMS','120000')
])

In [14]:
spark = SparkSession \
    .builder \
 .config(conf=config) \
    .appName("MongoDB-rewards-history") \
    .master("spark://172.23.149.210:7077") \
    .getOrCreate()

23/01/16 22:14:51 WARN Utils: Your hostname, cardano-druid resolves to a loopback address: 127.0.0.1; using 172.23.149.210 instead (on interface ens3)
23/01/16 22:14:51 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ubuntu/.ivy2/cache
The jars for the packages stored in: /home/ubuntu/.ivy2/jars
org.mongodb.spark#mongo-spark-connector added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-5c7deeb4-a416-42b4-89e7-65372bf5b6b7;1.0
	confs: [default]
	found org.mongodb.spark#mongo-spark-connector;10.0.2 in central
	found org.mongodb#mongodb-driver-sync;4.5.1 in central
	[4.5.1] org.mongodb#mongodb-driver-sync;[4.5.0,4.5.99)
	found org.mongodb#bson;4.5.1 in central
	found org.mongodb#mongodb-driver-core;4.5.1 in central
:: resolution report :: resolve 2196ms :: artifacts dl 5ms
	:: modules in use:
	org.mongodb#bson;4.5.1 from central in [default]
	org.mongodb#mongodb-driver-core;4.5.1 from central in [default]
	org.mongodb#mongodb-driver-sync;4.5.1 from central in [default]
	org.mongodb.spark#mongo-spark-connector;10.0.2 from central in [default]
	---------------------------------------------------------------------
	|                  |  

# Tables needed

- reward
- pool_hash
- stake_address

In [15]:
reward = spark.read.format("mongodb") \
 .option('spark.mongodb.connection.uri', 'mongodb://172.23.149.210:27017') \
   .option('spark.mongodb.database', 'cardano_bronze') \
   .option('spark.mongodb.collection', 'reward_temporary') \
 .option('spark.mongodb.read.readPreference.name', 'primaryPreferred') \
 .option('spark.mongodb.change.stream.publish.full.document.only','true') \
   .option("forceDeleteTempCheckpointLocation", "true") \
   .load()

In [16]:
pool_hash = spark.read.format("mongodb") \
 .option('spark.mongodb.connection.uri', 'mongodb://172.23.149.210:27017') \
   .option('spark.mongodb.database', 'cardano_bronze') \
   .option('spark.mongodb.collection', 'node.public.pool_hash') \
 .option('spark.mongodb.read.readPreference.name', 'primaryPreferred') \
 .option('spark.mongodb.change.stream.publish.full.document.only','true') \
   .option("forceDeleteTempCheckpointLocation", "true") \
   .load()

In [17]:
stake_address = spark.read.format("mongodb") \
 .option('spark.mongodb.connection.uri', 'mongodb://172.23.149.210:27017') \
   .option('spark.mongodb.database', 'cardano_bronze') \
   .option('spark.mongodb.collection', 'node.public.stake_address') \
 .option('spark.mongodb.read.readPreference.name', 'primaryPreferred') \
 .option('spark.mongodb.change.stream.publish.full.document.only','true') \
   .option("forceDeleteTempCheckpointLocation", "true") \
   .load()

In [18]:
reward.createOrReplaceTempView("reward")
pool_hash.createOrReplaceTempView("pool_hash")
stake_address.createOrReplaceTempView("stake_address")

# Get the reward history per address:
Meaning: retrieve the rewards by epoch, grouping by address. Delegated pools are also reported in the query.

In [19]:
reward_history_address = "select stake_address.id, stake_address.view, reward.earned_epoch, \
pool_hash.view as delegated_pool, reward.amount as lovelace \
    from reward \
    inner join stake_address on reward.addr_id = stake_address.id \
    inner join pool_hash on reward.pool_id = pool_hash.id \
    order by earned_epoch desc;"

In [20]:
reward_history_address_result = spark.sql(reward_history_address)
reward_history_address_result.createOrReplaceTempView("reward_history_address")

In [21]:
## Write in the SILVER db, collection rewards_by_address. This is the first step of processing the data to find insights in the rewards history.
## Note: the writing takes place as temporary table on Spark
reward_history_address_result.write.format("mongodb") \
	.option('spark.mongodb.connection.uri', 'mongodb://172.23.149.210:27017') \
  	.mode("append") \
    .option('spark.mongodb.database', 'cardano_silver') \
  	.option('spark.mongodb.collection', 'rewards_history') \
  	.option("forceDeleteTempCheckpointLocation", "true") \
  	.save()

23/01/16 22:17:09 WARN CaseInsensitiveStringMap: Converting duplicated key forcedeletetempcheckpointlocation into CaseInsensitiveStringMap.


# Intermediate import of rewards_history table

In [23]:
reward_history_address = spark.read.format("mongodb") \
 .option('spark.mongodb.connection.uri', 'mongodb://172.23.149.210:27017') \
   .option('spark.mongodb.database', 'cardano_silver') \
   .option('spark.mongodb.collection', 'rewards_history') \
 .option('spark.mongodb.read.readPreference.name', 'primaryPreferred') \
 .option('spark.mongodb.change.stream.publish.full.document.only','true') \
   .option("forceDeleteTempCheckpointLocation", "true") \
   .load()

In [24]:
reward_history_address.createOrReplaceTempView("reward_history_address")

### Gold database results

__2. Get the total rewards by epoch - rewards_by_epoch__

In [25]:
rewards_by_epoch = "select earned_epoch, sum(lovelace)/1000000 as tot_ADA \
from reward_history_address \
group by earned_epoch"

In [26]:
rewards_by_epoch = spark.sql(rewards_by_epoch)

In [27]:
rewards_by_epoch.write.format("mongodb") \
	.option('spark.mongodb.connection.uri', 'mongodb://172.23.149.210:27017') \
  	.mode("overwrite") \
    .option('spark.mongodb.database', 'cardano_gold') \
  	.option('spark.mongodb.collection', 'rewards_by_epoch') \
  	.option("forceDeleteTempCheckpointLocation", "true") \
  	.save()

23/01/16 23:28:49 WARN CaseInsensitiveStringMap: Converting duplicated key forcedeletetempcheckpointlocation into CaseInsensitiveStringMap.


__3. Get the total rewards by pool - rewards_by_pool__

In [28]:
rewards_by_pool = "select delegated_pool, sum(lovelace)/1000000 as tot_ADA \
from reward_history_address \
group by delegated_pool"

In [29]:
rewards_by_pool = spark.sql(rewards_by_pool)

In [30]:
rewards_by_pool.write.format("mongodb") \
	.option('spark.mongodb.connection.uri', 'mongodb://172.23.149.210:27017') \
  	.mode("overwrite") \
    .option('spark.mongodb.database', 'cardano_gold') \
  	.option('spark.mongodb.collection', 'rewards_by_pool') \
  	.option("forceDeleteTempCheckpointLocation", "true") \
  	.save()

23/01/16 23:55:54 WARN CaseInsensitiveStringMap: Converting duplicated key forcedeletetempcheckpointlocation into CaseInsensitiveStringMap.


__4. Proportion of top stakers - distribution_whales__

In [31]:
distribution_whales = "select view, sum(lovelace)/1000000 as tot_ADA \
from reward_history_address \
group by view \
order by tot_ADA desc \
limit 10"

In [32]:
distribution_whales = spark.sql(distribution_whales)

In [33]:
distribution_whales.write.format("mongodb") \
	.option('spark.mongodb.connection.uri', 'mongodb://172.23.149.210:27017') \
  	.mode("overwrite") \
    .option('spark.mongodb.database', 'cardano_gold') \
  	.option('spark.mongodb.collection', 'distribution_whales') \
  	.option("forceDeleteTempCheckpointLocation", "true") \
  	.save()

23/01/16 23:58:24 WARN CaseInsensitiveStringMap: Converting duplicated key forcedeletetempcheckpointlocation into CaseInsensitiveStringMap.


In [34]:
# the full REWARD table is now required to compute the rewards by type and sum of ADA rewards by type subqueries
reward = spark.read.format("mongodb") \
 .option('spark.mongodb.connection.uri', 'mongodb://172.23.149.210:27017') \
   .option('spark.mongodb.database', 'cardano_bronze') \
   .option('spark.mongodb.collection', 'node.public.reward') \
 .option('spark.mongodb.read.readPreference.name', 'primaryPreferred') \
 .option('spark.mongodb.change.stream.publish.full.document.only','true') \
   .option("forceDeleteTempCheckpointLocation", "true") \
   .load()

In [35]:
reward.createOrReplaceTempView("reward")

__5. Count how many rewards by type__

In [36]:
rewards_by_type = "select count(*),type from reward group by type"

In [37]:
rewards_by_type = spark.sql(rewards_by_type)

In [38]:
#rewards_by_type.write.format("mongodb") \
#	.option('spark.mongodb.connection.uri', 'mongodb://172.23.149.210:27017') \
#  	.mode("overwrite") \
#    .option('spark.mongodb.database', 'cardano_gold') \
#  	.option('spark.mongodb.collection', 'rewards_by_type') \
#  	.option("forceDeleteTempCheckpointLocation", "true") \
#  	.save()

23/01/17 00:05:41 WARN CaseInsensitiveStringMap: Converting duplicated key forcedeletetempcheckpointlocation into CaseInsensitiveStringMap.
23/01/17 00:15:50 WARN TaskSetManager: Lost task 276.0 in stage 24.0 (TID 1522) (172.23.149.210 executor 0): com.mongodb.MongoSocketReadException: Exception receiving message
	at com.mongodb.internal.connection.InternalStreamConnection.translateReadException(InternalStreamConnection.java:707)
	at com.mongodb.internal.connection.InternalStreamConnection.receiveMessageWithAdditionalTimeout(InternalStreamConnection.java:579)
	at com.mongodb.internal.connection.InternalStreamConnection.receiveCommandMessageResponse(InternalStreamConnection.java:415)
	at com.mongodb.internal.connection.InternalStreamConnection.sendAndReceive(InternalStreamConnection.java:342)
	at com.mongodb.internal.connection.UsageTrackingInternalConnection.sendAndReceive(UsageTrackingInternalConnection.java:116)
	at com.mongodb.internal.connection.DefaultConnectionPool$PooledConnecti

__6. Sum ADA rewards by type__

In [39]:
sum_rewards_by_type = "select sum(amount)/1000000 as ADA_amount,type from reward group by type"

In [40]:
sum_rewards_by_type = spark.sql(sum_rewards_by_type)

In [41]:
sum_rewards_by_type.write.format("mongodb") \
	.option('spark.mongodb.connection.uri', 'mongodb://172.23.149.210:27017') \
  	.mode("overwrite") \
    .option('spark.mongodb.database', 'cardano_gold') \
  	.option('spark.mongodb.collection', 'ADA_rewards_by_type') \
  	.option("forceDeleteTempCheckpointLocation", "true") \
  	.save()

23/01/17 00:16:46 WARN CaseInsensitiveStringMap: Converting duplicated key forcedeletetempcheckpointlocation into CaseInsensitiveStringMap.


In [42]:
spark.stop()

In [43]:
# update the old checkpoints with new ones, based on current document count
reward_query = { "collection": "reward" }
new_reward_count = { "$set": { "last_index": count_reward } }

last_ind.update_one(reward_query, new_reward_count)

In [44]:
# Grab Currrent Time After Running the Code
end = time.time()

#Subtract Start Time from The End Time
total_time = end - start
print("\n"+ str(total_time))


8391.45000576973
